<a href="https://colab.research.google.com/github/Tae-Hwanii/SEMI-SUPERVISED-CLASSIFICATION-WITH-GRAPH-CONVOLUTIONAL-NETWORKS/blob/main/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

class GraphConvolution(Module):
    def __init__(self, in_features, out_features, bias=True):
    # Module 클래스를 상속받는 GraphConvolution 클래스의 생성자
    # in_feauters : 입력 특징의 수, out_features : 출력 특성의 수, bias : 편향 사용 여부
        super(GraphConvolution, self).__init__()
        # 상위 클래스(Module)의 생성자를 호출하여 초기화
        self.in_features = in_features
        # 입력 특성 수를 클래스 속성으로 저장
        self.out_features = out_features
        # 출력 특성의 수를 클래스 속성으로 저장
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        # 레이어의 가중치 행렬을 정의하고 Parameter로 감싸서 모델 파라미터로 만듦
        # 크기는 (입력 특성의 수, 출력 특성의 수)
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
            # 편향 사용 여부에 따라 편향 항을 정의하고 Parameter로 감싸서 모델 파라미터로 만듦
            # 크기는 (출력 특성의 수)
        else:
            self.register_parameter('bias', None)
            # 편향을 사용하지 않는 경우, bias를 None으로 등록하여 레이어에서 사용되지 않도록 힘
        self.reset_parameters()
        # reset_parameters() 함수를 호출하여 가중치와 편향을 초기화

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        # 가중치 초기화에 사용할 표준 편차를 계산
        self.weight.data.uniform_(-stdv, stdv)
        # 가중치 행렬을 -stdv에서 stdv 사이의 균일한 랜덤 값으로 초기화
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
            # 편향이 존재하는 경우, 편향을 -stdv에서 stdv 사이의 균일한 랜덤 값으로 초기화


    def forward(self, input, adj):
        # 그래프 합성곱 레이어의 순방향 연산을 정의하는 메서드
        # input : 입력 데이터 (노드의 특성 행렬)
        # adj : 그래프의 연결성 정보를 나타내는 희소 행렬 (인접 행렬)
        support = torch.mm(input, self.weight)
        # 입력 데이터와 가중치 행렬을 곱하여 support를 계산
        # support는 입력 데이터에 가중치를 적용한 결과로, 각 노드에 대한 새로운 특성 행렬
        output = torch.spmm(adj, support)
        # 희소 행렬과 support를 곱하여 그래프 합성곱 연산을 수행하고 output을 얻음
        # output은 그래프에 따라 연결된 이웃 노드의 정보를 사용한 결과로, 각 노드에 대한 새로운 특성 행렬
        if self.bias is not None:
            return output + self.bias
            # 편향(bias)이 존재하는 경우, 결과에 편향을 더하고 반환
        else:
            return output
            # 편향이 존재하지 않는 경우, 그래프 합성곱 연산 결과만 반환


    def __repr__(self):
        # 객체를 문자열로 표현하는 메서드 정의
        return self.__class__.__name__ + ' (' \
                + str(self.in_features) + ' -> ' \
                + str(self.out_features) + ')'
        # 객체의 클래스 이름과 입력 및 출력 특성의 수를 포함한 문자열 반환

In [64]:
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        # GCN 모델의 생성자
        # nfeat : 입력 특성의 수, nhid : 은닉층의 특성 수, nclass : 출력 클래스 수, dropout : 드롭아웃 확률
        super(GCN, self).__init__()
        # 상위 클래스(nn.Moudle)의 생성자 호출

        self.gc1 = GraphConvolution(nfeat, nhid)
        # 첫 번째 그래프 합성곱 레이어 (입력 특성 -> 은닉층)
        self.gc2 = GraphConvolution(nhid, nclass)
        # 두 번째 그래프 합성곱 레이어 (은닉층 -> 출력 클래스)
        self.dropout = dropout
        # 드롭아웃 확률

    def forward(self, x, adj):
        # 순방향 연산 메서드
        x = F.relu(self.gc1(x, adj))
        # 첫 번째 그래프 합성곱 레이어를 통과한 후 ReLU 활성화 함수 적용
        x = F.dropout(x, self.dropout, training=self.training)
        # 드롭아웃 적용
        x = self.gc2(x, adj)
        # 두 번째 그래프 합성곱 레이어를 통과
        return F.log_softmax(x, dim=1)
        # 소프트맥스 함수를 사용하여 출력을 확률 분포로 변환하고, 로그 확률값 반환

In [65]:
import numpy as np
import scipy.sparse as sp
import torch

def encode_onehot(labels):
    # 레이블을 원-핫 인코딩하는 함수 정의
    # labels : 입력으로 주어진 레이블 리스트
    classes = set(labels)
    # 주어진 레이블 리스트에서 고유한 클래스(레이블)를 추출하여 집합(set)으로 만듦
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    # 각 클래스(레이블)를 원-핫 인코딩으로 나타내는 딕셔너리(class_dict) 생성
    # np.identity(len(classes))는 크기가 클래스 수와 같은 단위 행렬을 생성
    # enumerate(classes)를 통해 클래스(레이블)를 순회하며 해당 클래스에 해당하는 단위 행렬 행을 추출
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    # labels 리스트의 각 레이블을 classes_dict에서 찾아 원-핫 인코딩으로 반환
    # 반환된 결과를 Numpy 배열로 변환하고 데이터 타입을 int32로 설정
    return labels_onehot
    # 원-핫 인코딩된 레이블을 반환

def load_data(path="/content/drive/MyDrive/cora/", dataset="cora"):
    # 데이터를 로드하고 전처리하는 함수 정의
    # path : 데이터 파일이 위치한 경로, dataset : 데이터셋 이름
    print('Loading {} dataset. . .'.format(dataset))
    # 데이터셋 로딩 메시지 출력

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), dtype=np.dtype(str))
    # 데이터셋 파일에서 데이터를 읽어와서 numpy 배열로 저장
    # 데이터는 공백으로 구분되어 있고, dtype(str)을 사용하여 문자열로 읽음
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    # 데이터에서 특성(features) 부분을 추출하여 희소 특성 행렬(csr_matrix)로 변환
    # 1번 열부터 마지막에서 1번 열까지를 선택하며, 데이터 타입을 float32로 설정
    labels = encode_onehot(idx_features_labels[:, -1])
    # 데이터에서 레이블(Label) 부분을 추출하여 원-핫 인코딩된 레이블로 변환
    # encode_onehot 함수를 사용하여 레이블을 원-핫 인코딩

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    # 데이터에서 인덱스 정보를 추출하여 NumPy 배열로 저장
    # 인덱스는 정수형(int32)으로 저장됨
    idx_map = {j: i for i, j in enumerate(idx)}
    # 인덱스를 매핑하는 딕셔너리(idx_map) 생성
    # 기존 인덱스를 새로운 인덱스로 매핑하는 역할을 함
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset), dtype=np.int32)
    # 그래프의 엣지 정보를 포함한 텍스트 파일을 읽어와서 NumPy 배열로 저장
    # 데이터 타입은 정수형(int32)으로 설정
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
    # 엣지 정보의 인덱스를 새로운 인덱스로 변환
    # idx_map을 사용하여 엣지 정보의 인덱스를 새로운 인덱스로 매핑
    # 엣지 정보의 배열을 펼친 다음, 새로운 인덱스로 변환하고 다시 원래 형태로 변환
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(labels.shape[0], labels.shape[0]),dtype=np.float32)
    # 희소 고유값(coo) 행렬을 생성하여 그래프의 인접 행렬(adjacenct matrix)을 나타냄
    # (edges.shape[0])은 엣지의 수를 나타냄
    # (edges[:, 0], edges[:, i])은 각 엣지의 연결된 노드 인덱스를 나타냄
    # shape=(labels.shape[0], labels.shape[0])은 인접 행렬의 크기를 설정
    # dtype=np.float32은 데이터 타입을 부동소수점(float32)으로 설정

    # build symmetric adjacency matrix
    adj = adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    # 인접 행렬(adj)을 대칭으로 만듦
    # adj.T는 인접 행렬을 전치한 것이며, '>' 연산자로 대칭성을 검사
    # multiply 함수를 사용하여 대칭된 부분만 남기고, 대칭되지 않은 부분을 0으로 만듦

    features = normalize(features)
    # 특성 행렬(features)을 정규화(normalize)
    adj = normalize(adj + sp.eye(adj.shape[0]))
    # 대칭 인접 행렬(adj)에 단위 행렬(eye)을 더하고 다시 정규화
    # 이렇게 함으로써 자기 루프(self-loop)를 고려한 인접 행렬 생성

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)
    # 훈련, 검증, 테스트 데이터의 인덱스를 설정

    features = torch.FloatTensor(np.array(features.todense()))
    # 특성 행렬을 NumPy 배열로 변환하고, PyTorch의 Float Tensor로 변환
    labels = torch.LongTensor(np.where(labels)[1])
    # 레이블을 NumPy 배열로 변환하고, PyTorch의 LongTensor로 변환
    adj = sparse_mx_to_torch_sparse_tensor(adj)
    # 인접 행렬을 PyTorch의 희소 텐서(Sparse Tensor)로 변환

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)
    # 인덱스들을 PyTorch의 LongTensor로 변환

    return adj, features, labels, idx_train, idx_val, idx_test
    # 준비된 데이터를 반환

def normalize(mx):
    rowsum = np.array(mx.sum(1))
    # 행렬의 각 행(row)의 합을 계산하여 배열(rowsum)로 저장
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    # 각 행의 합에 역수를 취하고, 0으로 나누는 경우(무한대 역수)에 대한 처리를 수행
    r_mat_inv = sp.diags(r_inv)
    # 대각 행렬(diagonal matrix)을 생성하고, 대각 성분에 역수 값을 포함
    mx = r_mat_inv.dot(mx)
    # 입력 행렬(mx)에 대각 행렬을 왼쪽에서 곱하여 행렬을 정규화
    return mx
    # 정규화된 행렬을 반환

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    # 모델의 출력(output)에서 가장 큰 값의 인덱스(max(1)[1])를 선택하여 예측(preds)을 구함
    # 예측(preds)을 실제 레이블(labels)의 데이터 타입(type_as)으로 변환
    correct = preds.eq(labels).double()
    # 예측(preds)과 실제 레이블(labels)을 비교하여 일치하는 경우 1, 아닌 경우 0인 이진 값으로 변환
    correct = correct.sum()
    # 일치하는 값을 합산하여 정확한 예측의 수를 계산
    return correct / len(labels)
    # 정확한 예측의 수를 전체 레이블(labels)의 수로 나누어 정확도(accuracy)를 계산

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
     # 희소 행렬(sparse_mx)을 COO(coordinate List) 형식으로 변환하고 데이터 타입을 float32로 변환
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
     # COO 형식의 희소 행렬을 PyTorch 텐서로 변환하기 위해 인덱스(indices), 값(values), 크기(shape)를 추출
    return torch.sparse.FloatTensor(indices, values, shape)
    # PyTorch의 희소 텐서(Sparse Tensor)를 생성하여 변환

In [66]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

#training setting

no_cuda = False
fastmode = False
seed = 42
epochs = 1000
lr = 0.01
weight_decay = 5e-4
hidden = 16
dropout = 0.5

np.random.seed(seed)
torch.manual_seed(seed)
# 실험의 재현성을 위해 난수 시드 설정
# NumPy 및 PyTorch의 난수 생성을 제어하기 위한 시드 설정

#Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

#Model and optimizer
model = GCN(nfeat = features.shape[1],                    # 입력 특성의 크기를 모델에 전달
            nhid = hidden,                           # 은닉층의 크기 설정 (hidden units)
            nclass = labels.max().item() + 1,             # 클래스 수를 레이블에서 결정
            dropout = dropout)                       # 드롭아웃 확률 설정
optimizer = optim.Adam(model.parameters(),              # Adam 최적화 알고리즘 사용
                       lr = lr,                      # 학습률(learning rate) 설정
                       weight_decay = weight_decay)  # 가중치 감쇠(L2 손실) 설정

def train(epoch):
    t = time.time()
    # 현재 학습 epoch의 시작 시간 기록
    model.train()
    # 모델 학습 모드로 설정
    optimizer.zero_grad()
    # 기울기 초기화
    output = model(features, adj)
    # 모델에 입력 데이터와 인접 행렬을 전달하여 출력 예측 계산
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    # 학습 데이터에 대한 손실(loss) 및 정확도(accuracy) 계산
    loss_train.backward()
    optimizer.step()
    # 역전파 및 모델 파라미터 업데이트

    if not fastmode:
        model.eval()
        output = model(features, adj)
        # 만약 'fastmode'가 비활성화된 경우, 검증 데이터에 대한 평가 수행

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    # 검증 데이터에 대한 손실(loss) 및 정확도(accuracy) 계산
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

def test():
    model.eval()
    # 모델을 평가 모드로 설정
    output = model(features, adj)
    # 모델을 사용하여 출력 예측 계산
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    # 테스트 데이터에 대한 손실(loss)과 정확도(accuracy) 계산
    print("Test set results",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

t_total = time.time()
# 학습 시간 측정을 위한 변수 설정
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time slapsed: {:.4f}s".format(time.time() - t_total))

test()

Loading cora dataset. . .
Epoch: 0001 loss_train: 1.9766 acc_train: 0.0786 loss_val: 1.9628 acc_val: 0.1233 time: 0.0248s
Epoch: 0002 loss_train: 1.9648 acc_train: 0.1357 loss_val: 1.9517 acc_val: 0.1267 time: 0.0251s
Epoch: 0003 loss_train: 1.9509 acc_train: 0.1500 loss_val: 1.9409 acc_val: 0.1267 time: 0.0264s
Epoch: 0004 loss_train: 1.9393 acc_train: 0.1571 loss_val: 1.9302 acc_val: 0.1267 time: 0.0250s
Epoch: 0005 loss_train: 1.9343 acc_train: 0.1286 loss_val: 1.9194 acc_val: 0.1267 time: 0.0258s
Epoch: 0006 loss_train: 1.9101 acc_train: 0.2714 loss_val: 1.9080 acc_val: 0.1867 time: 0.0247s
Epoch: 0007 loss_train: 1.9078 acc_train: 0.3071 loss_val: 1.8965 acc_val: 0.3433 time: 0.0252s
Epoch: 0008 loss_train: 1.8821 acc_train: 0.3643 loss_val: 1.8845 acc_val: 0.3533 time: 0.0260s
Epoch: 0009 loss_train: 1.8785 acc_train: 0.3214 loss_val: 1.8723 acc_val: 0.3567 time: 0.0332s
Epoch: 0010 loss_train: 1.8662 acc_train: 0.3429 loss_val: 1.8597 acc_val: 0.3567 time: 0.0247s
Epoch: 0011 lo